<a href="https://www.kaggle.com/code/rv1922/s4-e10-loan-status-prediction-catboost-97?scriptVersionId=200618561" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <p style="background-color: #90EE90; font-family:'Dancing Script', cursive; font-weight:bold; color:black; font-size:120%; text-align:center; border: 1px solid black; border-radius:10px; padding: 10px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);">Import Package</p>

In [1]:
import numpy as np
import pandas as pd 
import optuna
import plotly.graph_objects as go
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.simplefilter('ignore')

# <p style="background-color: #90EE90; font-family:'Dancing Script', cursive; font-weight:bold; color:black; font-size:120%; text-align:center; border: 1px solid black; border-radius:10px; padding: 10px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);">Read Data</p>

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e10/test.csv")
original = pd.read_csv("/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv")
submission = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')

In [3]:
train.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [4]:
original.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


# <p style="background-color: #90EE90; font-family:'Dancing Script', cursive; font-weight:bold; color:black; font-size:120%; text-align:center; border: 1px solid black; border-radius:10px; padding: 10px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);">Encoding Feature and Handling Null Values</p>

In [5]:
original['person_emp_length'] = original['person_emp_length'].fillna(original['person_emp_length'].median())
original['loan_int_rate'] = original['loan_int_rate'].fillna(original['loan_int_rate'].median())

In [6]:
train = pd.concat([train, original], ignore_index = True)
train['id'] = np.arange(91226)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91226 entries, 0 to 91225
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          91226 non-null  int64  
 1   person_age                  91226 non-null  int64  
 2   person_income               91226 non-null  int64  
 3   person_home_ownership       91226 non-null  object 
 4   person_emp_length           91226 non-null  float64
 5   loan_intent                 91226 non-null  object 
 6   loan_grade                  91226 non-null  object 
 7   loan_amnt                   91226 non-null  int64  
 8   loan_int_rate               91226 non-null  float64
 9   loan_percent_income         91226 non-null  float64
 10  cb_person_default_on_file   91226 non-null  object 
 11  cb_person_cred_hist_length  91226 non-null  int64  
 12  loan_status                 91226 non-null  int64  
dtypes: float64(3), int64(6), object

In [8]:
train['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}, inplace=True)
train['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}, inplace=True)
train['loan_grade'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}, inplace=True)
train['cb_person_default_on_file'].replace({'N': 0, 'Y': 1}, inplace=True)
train['person_emp_length'] = train['person_emp_length'].astype(int)
train['loan_int_rate'] = (train['loan_int_rate'] * 100).astype(int)
train['loan_percent_income'] = (train['loan_percent_income'] * 100).astype(int)
train["loantoincome"] = (((train["loan_amnt"] / train["person_income"])) * 1000000).astype(int)

In [9]:
test['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}, inplace=True)
test['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}, inplace=True)
test['loan_grade'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}, inplace=True)
test['cb_person_default_on_file'].replace({'N': 0, 'Y': 1}, inplace=True)
test['person_emp_length'] = test['person_emp_length'].astype(int)
test['loan_int_rate'] = (test['loan_int_rate'] * 100).astype(int)
test['loan_percent_income'] = (test['loan_percent_income'] * 100).astype(int)
test["loantoincome"] = (((test["loan_amnt"] / test["person_income"])) * 1000000).astype(int)

In [10]:
train.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,loantoincome
0,0,37,35000,0,0,0,1,6000,1149,17,0,14,0,171428
1,1,22,56000,2,6,1,2,4000,1335,7,0,2,0,71428
2,2,29,28800,2,8,2,0,6000,890,21,0,10,0,208333
3,3,30,70000,0,14,3,1,12000,1111,17,0,5,0,171428
4,4,22,60000,0,2,1,0,6000,692,10,0,3,0,100000


# <p style="background-color: #90EE90; font-family:'Dancing Script', cursive; font-weight:bold; color:black; font-size:120%; text-align:center; border: 1px solid black; border-radius:10px; padding: 10px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);">Train Model</p>

In [11]:
X = train.drop(['loan_status'], axis=1)
y = train['loan_status']

In [12]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)
    iterations = trial.suggest_int('iterations', 100, 2000)
    depth = trial.suggest_int('depth', 4, 10)
    random_strength = trial.suggest_loguniform('random_strength', 1e-9, 10)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []
    
    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        model = CatBoostClassifier(
            learning_rate=learning_rate,
            iterations=iterations,
            depth=depth,
            random_strength=random_strength,
            l2_leaf_reg=l2_leaf_reg,
            loss_function='Logloss',
            eval_metric='AUC',
            task_type='GPU',  # Change to 'CPU' if GPU is not available
            random_seed=42,
            verbose=False
        )
        
        train_pool = Pool(X_train, y_train)
        valid_pool = Pool(X_valid, y_valid)

        model.fit(train_pool, eval_set=valid_pool)

        pred_valid = model.predict_proba(valid_pool)[:, 1]

        auc = roc_auc_score(y_valid, pred_valid)
        aucs.append(auc)
    
    return np.mean(aucs)

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  

print("Best hyperparameters: ", study.best_params)
print("Best AUC: ", study.best_value)

[I 2024-10-12 14:25:24,627] A new study created in memory with name: no-name-3b3eddd4-570d-47d5-bca6-3b35d2e25d21
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2024-10-12 14:26:00,414] Trial 0 finished with value: 0.9346532828892574 and parameters: {'learning_rate': 0.03395499023341701, 'iterations': 150, 'depth': 7, 'random_strength': 0.0057059935072808834, 'l2_leaf_reg': 0.4545381616953975}. Best is trial 0 with value: 0.9346532828892574.
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC

Best hyperparameters:  {'learning_rate': 0.06763295577624828, 'iterations': 1176, 'depth': 5, 'random_strength': 6.556812310120627e-08, 'l2_leaf_reg': 3.4735978957981715}
Best AUC:  0.9558111163266625
